In [3]:
import os
import pandas as pd
from datetime import datetime

# import tensorflow as tf
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator


def extract_scalars_from_event_file(event_file):
    event_acc = EventAccumulator(event_file)
    event_acc.Reload()
    

    tags = event_acc.Tags()["scalars"]

    all_scalars = {}
    for tag in tags:
        events = event_acc.Scalars(tag)
        values = [event.value for event in events]
        steps = [event.step for event in events]

        timestamps = [pd.to_datetime(event.wall_time, unit='s') for event in events]
        all_scalars[tag] = pd.DataFrame({"step": steps, "value": values, "timestamp": timestamps})

    return all_scalars


files_t = []
def event_files_to_dataframe(directory):
    all_scalars_dfs = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.startswith("events.out.tfevents"):
                event_file = os.path.join(root, file)

                file_from = pd.to_datetime(file.split(".")[3], unit='s')
                files_t.append(file_from)

                # print(file_from)
                if file_from < date_obj:
                    continue

                scalars = extract_scalars_from_event_file(event_file)
                for tag, df in scalars.items():
                    df["tag"] = tag
                    all_scalars_dfs.append(df)

    all_scalars_df = pd.concat(all_scalars_dfs, ignore_index=True)
    return all_scalars_df

df_all = []
for i in range(8):
    # Directory containing your event files
    log_dir = f"/home/<User>/repos/csg_is/calculating_scores/CFL/8Nodes_non_iid/fedstellar_CFL_17_07_2024_21_50_13/metrics/participant_{i}"
    date_str = "17/07/2024 19:50:13 ".rstrip(" ") # minus two hours for timezone

    date_obj = datetime.strptime(date_str, "%d/%m/%Y %H:%M:%S")

    df = event_files_to_dataframe(log_dir)
    print(df.head())

    df.sort_values(by="timestamp", inplace=True)

    files_t.sort(reverse=True)
    print(files_t[0], files_t[-1])

    df[df["tag"]=='Test/Loss']["timestamp"].max()

    test_df = df[df["tag"].str.startswith("Test/")].groupby("tag").last().copy()
    print(test_df)
    test_df["participant"] = f"participant_{i}"
    df_all.append(test_df)

   step      value                     timestamp                    tag
0  4256  26.900000 2024-07-17 20:02:22.631595264  Resources/CPU_percent
1  4266  65.099998 2024-07-17 20:02:32.755728640  Resources/CPU_percent
2  4276  87.500000 2024-07-17 20:02:42.775500288  Resources/CPU_percent
3  4286  88.199997 2024-07-17 20:02:52.833567744  Resources/CPU_percent
4  4297  81.300003 2024-07-17 20:03:03.018553856  Resources/CPU_percent
2024-07-17 20:04:54 2024-07-17 19:50:25
                step     value                     timestamp
tag                                                         
Test/Accuracy   5188  0.521580 2024-07-17 20:04:51.950301696
Test/F1Score    5188  0.429179 2024-07-17 20:04:51.950332672
Test/Loss       5188  2.955770 2024-07-17 20:04:51.950270720
Test/Precision  5188  0.415971 2024-07-17 20:04:51.950312960
Test/Recall     5188  0.521580 2024-07-17 20:04:51.950323200
   step      value                     timestamp                    tag
0   143  40.200001 2024-07-17

In [4]:
concat_df = pd.concat(df_all)
concat_df

,step,value,timestamp,participant
tag,,,,
Test/Accuracy,5188,0.521580,2024-07-17 20:04:51.950301696,participant_0
Test/F1Score,5188,0.429179,2024-07-17 20:04:51.950332672,participant_0
Test/Loss,5188,2.955770,2024-07-17 20:04:51.950270720,participant_0
Test/Precision,5188,0.415971,2024-07-17 20:04:51.950312960,participant_0
Test/Recall,5188,0.521580,2024-07-17 20:04:51.950323200,participant_0
Test/Accuracy,5305,0.549592,2024-07-17 20:05:04.719470080,participant_1
Test/F1Score,5305,0.521235,2024-07-17 20:05:04.719502848,participant_1
Test/Loss,5305,2.261058,2024-07-17 20:05:04.719428608,participant_1
Test/Precision,5305,0.588786,2024-07-17 20:05:04.719481856,participant_1


In [5]:
concat_df.groupby("tag").mean().round(4)

/tmp/ipykernel_29270/4276830878.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  concat_df.groupby("tag").mean().round(4)


,step,value
tag,,
Test/Accuracy,5269.0,0.6652
Test/F1Score,5269.0,0.6265
Test/Loss,5269.0,1.6782
Test/Precision,5269.0,0.6531
Test/Recall,5269.0,0.6652


In [6]:
concat_df.groupby("tag").std().round(2)

/tmp/ipykernel_29270/582865284.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.std is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  concat_df.groupby("tag").std().round(2)


,step,value
tag,,
Test/Accuracy,51.65,0.19
Test/F1Score,51.65,0.22
Test/Loss,51.65,1.16
Test/Precision,51.65,0.22
Test/Recall,51.65,0.19


In [ ]:
test_results = concat_df.groupby("tag").mean().round(4)
test_results[test_results.index.str.startswith("Test/")]